* [About this Dataset](#about)
* [EDA](#eda)
* [Data preparation](#dataprep)
* [Modeling](#modeling)
    * [Meta Learners](#metalearners)
        * [T-Learners](#tlearners)
        * [S-Learners](#slearners)
    * [Uplift Trees](#utrees)
* [Conclusions](#conclusions)

# About this Dataset <a class="anchor" id="about"></a>

  The dataset was created by The Criteo AI Lab .The dataset consists of 13M rows, each one representing a user with 12 features, a treatment indicator and 2 binary labels (visits and conversions). Positive labels mean the user visited/converted on the advertiser website during the test period (2 weeks). The global treatment ratio is 84.6%. It is usual that advertisers keep only a small control population as it costs them in potential revenue.  
  
Following is a detailed description of the features:  
  
- f0, f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11: feature values (dense, float)
- treatment: treatment group (1 = treated, 0 = control)
- conversion: whether a conversion occured for this user (binary, label)
- visit: whether a visit occured for this user (binary, label)
- exposure: treatment effect, whether the user has been effectively exposed (binary)

In [3]:
!pip install causalml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.8 MB/s eta 0:00:0000:010:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.7/278.7 kB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for causalml: filename=causalml-0.15.2-cp37-cp37m-linux_x86_64.whl size=6428251 sha256=9ff47754c59620f7b6923a5763049e9f75aa17d6062a7b1ff107dd3ef69e1973
  Stored in directory: /root/.cache/pip/wheels/77/7b/df/ae934d53609fe0cd61502f11c72ed039222b98e08cdebe2a06
  Created wheel for pydotplus: filename=pydotplus-2.0.2-py3-none-any.whl size=24575 sha256=c370ec6be738007d5467da9d9e57b30911fa908b0e1ffdbe7bed87e6da23eae0
  Stored in directory: /root/.cache/pip/wheels/1e/7b/04/7387cf6cc9e48b4a96e361b0be812f0708b394b821bf8c9c50
Successfully built causalml pydotplus


In [4]:
!pip install scikit-uplift


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.2 MB/s eta 0:00:00


In [5]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
# from sklift.metrics import uplift_at_k, uplift_auc_score, qini_auc_score, weighted_average_uplift
# from sklift.viz import plot_uplift_preds
# from sklift.models import SoloModel, TwoModels
from sklearn.base import clone
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from causalml.inference.tree import UpliftRandomForestClassifier, UpliftTreeClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot
from IPython.display import Image

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import shap
def prepare_data(df, feature_cols, treatment_col='treatment', target_col='conversion', 
                 test_size=0.2, random_state=42):
    """
    Prepares data for uplift modeling with binary treatment/control setup
    """
    # Split data into features, treatment and target
    X = df[feature_cols]
    y = df[target_col]
    t = df[treatment_col]

    # Train-test split with stratification on treatment
    X_train, X_test, t_train, t_test, y_train, y_test = train_test_split(
        X, t, y, test_size=test_size, random_state=random_state, stratify=t
    )

    return {
        't_train': t_train, 't_test': t_test,'X_train': X_train, 'X_test': X_test,'y_train': y_train, 'y_test': y_test
    }

# Example usage
feature_cols = [f'f{i}' for i in range(12)]
df = pd.read_csv('../input/uplift-modeling/criteo-uplift-v2.1.csv')
data = prepare_data(df, feature_cols)

# Access prepared data
X_train = data['X_train']
X_test = data['X_test']
t_train = data['t_train']
t_test = data['t_test']
y_train = data['y_train']
y_test = data['y_test']


In [17]:
from causalml.inference.meta import BaseSRegressor
from causalml.inference.tree import UpliftRandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor
from xgboost import XGBRegressor

# Initialize and train S-Learner
learner_s = BaseSRegressor(learner=XGBRegressor(random_state=42))
learner_s_result = learner_s.fit_predict(X=X_train, treatment=t_train, y=y_train)

print(learner_s_result)


[[8.54994287e-05]
 [1.00145233e-04]
 [3.63811385e-03]
 ...
 [1.30322325e-04]
 [8.54995669e-05]
 [1.00145233e-04]]


# EDA <a class="anchor" id="eda"></a>

In [ ]:
from sklearn.neural_network import MLPRegressor
from causalml.inference.meta import BaseSRegressor

# Define the neural network regressor
nn = MLPRegressor(hidden_layer_sizes=(35, 25, 10, 5),
                  learning_rate_init=0.01,
                  early_stopping=True,
                  random_state=1)

# Plug it into the S-Learner
learner_s = BaseSRegressor(learner=nn)
learner_s_result = learner_s.fit_predict(X=X_train, treatment=t_train, y=y_train)


In [ ]:
# ITE
nn_ite = nn.fit_predict(X_train, t_train, y_train)

In [ ]:
# Feature importance using permutation
nn.get_importance(X=X_train, tau=nn_ite, method='permutation', features=feature_names, random_state=42)

Looks like the most effective uplift method for this dataset (from tested here) is uplift tree with overall uplift 0.03 (0.031 for top 30%). Among meta learners S-Learners perform better than T-Learners (with a S-Learner based on LGBMClassifier as a leader).